In [1]:
''' Import libraries '''
import numpy as np
import laspy
import tqdm
import os

In [2]:
''' Display data structure information '''
# Specify file path
file_path = [r"Area0_230713_031852_S0.laz", r"Area1_230724_065805_S0.laz", r"Area2_230724_054607_S0.laz", r"Area3_230724_011027_S0.laz"][1]

# Import data
with laspy.open(file_path, mode='r')  as las:

    # Print public header
    public_header = las.header
    print("Default Version:", public_header.version)
    print("Default Point Format:", public_header.point_format)
    print("Number of points:", public_header.point_count)
    print("Scale Factor:", public_header.scales)
    print("Offset: [{}, {}, {}]".format(*[format(num, '.2f') for num in public_header.offsets]))
    print("Min Bounds: [{}, {}, {}]".format(*[format(num, '.2f') for num in public_header.min]))
    print("Max Bounds: [{}, {}, {}]".format(*[format(num, '.2f') for num in public_header.max]))

    print("-" * 50)

    # Print variabe length records (VLR)
    vlrs = las.header.vlrs
    for vlr in vlrs:

        print("-" * 50)
        
        print("User ID:", vlr.user_id)
        print("Record ID:", vlr.record_id)
        print("Description:", vlr.description)

        if vlr.record_id == 34735:
            print(vlr.parse_crs)

        elif vlr.record_id == 34736:
            print(vlr)

        elif vlr.record_id == 34737:
            print(vlr)

    # No extended variable length records (EVLRs)

Default Version: 1.2
Default Point Format: <PointFormat(3, 0 bytes of extra dims)>
Number of points: 332123203
Scale Factor: [0.00025 0.00025 0.00025]
Offset: [-18926.90, -59289.69, 29.79]
Min Bounds: [-19115.71, -59507.12, -8.74]
Max Bounds: [-18738.09, -59072.26, 68.33]
--------------------------------------------------
--------------------------------------------------
User ID: LASF_Projection
Record ID: 34735
Description: GeoKeyDirectoryTag (mandatory)
<bound method GeoKeyDirectoryVlr.parse_crs of <GeoKeyDirectoryVlr(25 geo_keys)>>
--------------------------------------------------
User ID: LASF_Projection
Record ID: 34736
Description: GeoDoubleParamsTag (optional)
<GeoDoubleParamsVlr([c_double(6378137.0), c_double(298.257222101), c_double(0.0), c_double(36.0), c_double(139.833333333333), c_double(0.0), c_double(0.0), c_double(0.9999)])>
--------------------------------------------------
User ID: LASF_Projection
Record ID: 34737
Description: GeoASCIIParamsTag (optional)
<GeoAsciiPa

In [3]:
''' Reduce size of laz randomly '''
# Specify file path
file_path = [r"Area0_230713_031852_S0.laz", r"Area1_230724_065805_S0.laz", r"Area2_230724_054607_S0.laz", r"Area3_230724_011027_S0.laz"][1]

# Import point cloud data
with laspy.open(file_path, mode='r')  as las:
    public_header = las.header
    n_points_total = public_header.point_count
s_chunk = 1000000
randomness = 1000
length = int((n_points_total // s_chunk + 1) * (s_chunk / randomness))
print("total iterations: {}".format(n_points_total // s_chunk + 1))
points_array = np.full((length, 6), np.nan)

current_index = 0
with laspy.open(file_path) as las:
    for chunk in tqdm.tqdm(las.chunk_iterator(s_chunk)):
        indices = np.random.choice(len(chunk), size=int(s_chunk/randomness), replace=False)
        s_chunk_use = int(s_chunk/randomness)
        points_array[current_index:current_index+s_chunk_use, 0] = chunk['X'][indices]
        points_array[current_index:current_index+s_chunk_use, 1] = chunk['Y'][indices]
        points_array[current_index:current_index+s_chunk_use, 2] = chunk['Z'][indices]
        points_array[current_index:current_index+s_chunk_use, 3] = chunk['red'][indices]
        points_array[current_index:current_index+s_chunk_use, 4] = chunk['green'][indices]
        points_array[current_index:current_index+s_chunk_use, 5] = chunk['blue'][indices]
        current_index += s_chunk_use

# Remove np.nan data points stored as result of chunk reading
points_array = points_array[~np.all(np.isnan(points_array), axis=1)]
n_sub_points = len(points_array)

# Create a new LAS object
public_header.point_count = n_sub_points
sub_las = laspy.LasData(public_header)

# Populate the LAS object with modified points_array
sub_las.points['X'] = points_array[:, 0]
sub_las.points['Y'] = points_array[:, 1]
sub_las.points['Z'] = points_array[:, 2]
sub_las.points['red'] = points_array[:, 3]
sub_las.points['green'] = points_array[:, 4]
sub_las.points['blue'] = points_array[:, 5]

# Export sub_las
new_file_path = file_path.replace(".laz", r"_random_1_" + str(randomness) + r".laz")
sub_las.write(new_file_path)

total iterations: 333


0it [00:00, ?it/s]

333it [01:12,  4.57it/s]


In [7]:
''' Split laz into  '''
# Specify file path
file_path = [r"Area0_230713_031852_S0.laz", r"Area1_230724_065805_S0.laz", r"Area2_230724_054607_S0.laz", r"Area3_230724_011027_S0.laz"][3]

points_array = np.empty((0, 6), dtype=float)

# Import point cloud data
with laspy.open(file_path, mode='r') as las:
    public_header = las.header

    new_public_header = laspy.header.LasHeader()
    new_public_header.version = public_header.version
    new_public_header.point_format = public_header.point_format
    new_public_header.scales = public_header.scales
    new_public_header.offsets = public_header.offsets
    new_public_header.vlrs = public_header.vlrs

    n_points_total = public_header.point_count
    chunk_size = 50_000_000
    print("total iterations: {}".format(n_points_total // chunk_size + 1))
    
    for chunk_index, points in enumerate(tqdm.tqdm(las.chunk_iterator(chunk_size))):
        chunk_points_array = np.column_stack((points['X'], points['Y'], points['Z'], points['red'], points['green'], points['blue']))
        n_sub_points = len(chunk_points_array)
        
        new_public_header.point_count = n_sub_points
        sub_las = laspy.LasData(new_public_header)

        # Populate the LAS object with modified points_array
        sub_las.points['X'] = chunk_points_array[:, 0]
        sub_las.points['Y'] = chunk_points_array[:, 1]
        sub_las.points['Z'] = chunk_points_array[:, 2]
        sub_las.points['red'] = chunk_points_array[:, 3]
        sub_las.points['green'] = chunk_points_array[:, 4]
        sub_las.points['blue'] = chunk_points_array[:, 5]

        # Export sub_las
        new_file_path = file_path.replace(".laz", r"_split_" + str(chunk_index) + r".laz")
        new_file_path = r"split_laz\\" + new_file_path
        sub_las.write(new_file_path)

total iterations: 30


30it [08:39, 17.31s/it]
